# mmmeld Validation Tests

This notebook contains parameterized validation tests for mmmeld.py. We'll test various scenarios and rules using different combinations of media sources.

In [45]:
import subprocess
import os
from IPython.display import Video

def run_mmmeld(params):
    output_file = f"test_output_{params[params.index('--output') + 1]}"
    cmd = ["python", "mmmeld.py", "--autofill", "--cleanup"] + params + ["--output", output_file]
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(" ".join(cmd)) 
    print(result.stdout)
    print(result.stderr)
    return result.returncode == 0, output_file

def display_output(output_file):
    if os.path.exists(output_file):
        return Video(output_file)
    else:
        return f"Output file {output_file} not found."

# Define media sources
local_audio_short = 'test_media/10_seconds.wav'  # 10 seconds
local_audio_long = 'test_media/30_seconds.mp3'  # 30 seconds
youtube_audio = 'https://www.youtube.com/watch?v=dQw4w9WgXcQ'  # Rick Astley - Never Gonna Give You Up
local_image1 = 'test_media/square.jpg' # square
local_image2 = 'test_media/16_9.png' # 16:9
local_video_short = 'test_media/5_seconds.mp4'  # 5 seconds
local_video_short2 = 'test_media/5_seconds_2.mp4'  # 5 seconds
local_video_med = 'test_media/15_seconds.mp4'  # 15 seconds
local_video_med2 = 'test_media/15_seconds2.mp4'  # 15 seconds
local_video_long = 'test_media/3_minutes.webm'  # 3 minutes
youtube_video = 'https://www.youtube.com/watch?v=9bZkp7q19f0'  # PSY - GANGNAM STYLE
bg_music = 'test_media/IndigoMirage.mp3'

# Text for text-to-speech generation
tts_text = "To be, or not to be, that is the question."

## Test 1: Single Image with Main Audio

Expected behavior: The image should be displayed for the entire duration of the audio.

In [46]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--output", "test1.mp4"])
print(f"Test 1 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image test_media/square.jpg --output test1.mp4 --output test_output_test1.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libope

## Test 2: Single Image without Main Audio

Expected behavior: The image should be displayed for 5 seconds.

In [47]:
success, output_file = run_mmmeld(["--image", local_image1, "--output", "test2.mp4"])
print(f"Test 2 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --image test_media/square.jpg --output test2.mp4 --output test_output_test2.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-med

## Test 3: Single Video with Main Audio (Video Shorter)

Expected behavior: The video should loop to match the audio duration.

In [48]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", local_video_short, "--output", "test3.mp4"])
print(f"Test 3 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/30_seconds.mp3 --image test_media/5_seconds.mp4 --output test3.mp4 --output test_output_test3.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-lib

## Test 4: Single Video with Main Audio (Video Longer)

Expected behavior: The video should be cut to match the audio duration.

In [49]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_video_med, "--output", "test4.mp4"])
print(f"Test 4 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image test_media/15_seconds.mp4 --output test4.mp4 --output test_output_test4.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-li

## Test 5: Single Video without Main Audio

Expected behavior: The video should play once with its own audio.

In [50]:
success, output_file = run_mmmeld(["--image", local_video_short2, "--output", "test5.mp4"])
print(f"Test 5 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --image test_media/5_seconds_2.mp4 --output test5.mp4 --output test_output_test5.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enabl

## Test 6: Multiple Videos with Main Audio (Total Video Time <= Audio Time)

Expected behavior: Videos should play in sequence, then loop.

In [51]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", f"{local_video_short},{local_video_med2}", "--output", "test6.mp4"])
print(f"Test 6 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/30_seconds.mp3 --image test_media/5_seconds.mp4,test_media/15_seconds2.mp4 --output test6.mp4 --output test_output_test6.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom -

## Test 7: Multiple Videos with Main Audio (Total Video Time > Audio Time)

Expected behavior: Videos should play in sequence, cut at audio end.

In [52]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_video_short},{local_video_med}", "--output", "test7.mp4"])
print(f"Test 7 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image test_media/5_seconds.mp4,test_media/15_seconds.mp4 --output test7.mp4 --output test_output_test7.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --

## Test 8: Multiple Videos without Main Audio

Expected behavior: Videos should play in sequence once.

In [53]:
success, output_file = run_mmmeld(["--image", f"{local_video_med},{local_video_short}", "--output", "test8.mp4"])
print(f"Test 8 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --image test_media/15_seconds.mp4,test_media/5_seconds.mp4 --output test8.mp4 --output test_output_test8.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg

## Test 9: Videos + Images with Main Audio (Total Video Time < Audio Time)

Expected behavior: Play videos once in sequence, distribute remaining time equally among images.

In [54]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", f"{local_video_short},{local_image1},{local_image2}", "--output", "test9.mp4"])
print(f"Test 9 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/30_seconds.mp3 --image test_media/5_seconds.mp4,test_media/square.jpg,test_media/16_9.png --output test9.mp4 --output test_output_test9.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --

## Test 10: Videos + Images with Main Audio (Total Video Time >= Audio Time)

Expected behavior: Distribute 5 seconds each to images, truncate sequence to fit audio.

In [55]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_video_med},{local_image1},{local_image2}", "--output", "test10.mp4"])
print(f"Test 10 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image test_media/15_seconds.mp4,test_media/square.jpg,test_media/16_9.png --output test10.mp4 --output test_output_test10.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid

## Test 11: Videos + Images without Main Audio

Expected behavior: Play videos in sequence, show each image for 5 seconds.

In [56]:
success, output_file = run_mmmeld(["--image", f"{local_video_short},{local_image1},{local_video_med2},{local_image2}", "--output", "test11.mp4"])
print(f"Test 11 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --image test_media/5_seconds.mp4,test_media/square.jpg,test_media/15_seconds2.mp4,test_media/16_9.png --output test11.mp4 --output test_output_test11.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enabl

## Test 12: Image + Video with Main Audio (Video Shorter)

Expected behavior: End of video anchored to end of audio, image shown first, filling remaining time.

In [57]:
success, output_file = run_mmmeld(["--audio", local_audio_long, "--image", f"{local_image1},{local_video_med}", "--output", "test12.mp4"])
print(f"Test 12 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/30_seconds.mp3 --image test_media/square.jpg,test_media/15_seconds.mp4 --output test12.mp4 --output test_output_test12.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --e

## Test 13: Image + Video with Main Audio (Video Longer)

Expected behavior: Minimum 5s for image, then video (cut off).

In [67]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_image1},{local_video_med}", "--output", "test13.mp4"])
print(f"Test 13 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image test_media/square.jpg,test_media/15_seconds.mp4 --output test13.mp4 --output test_output_test13.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --e

## Test 14: Image + Video without Main Audio

Expected behavior: Image shown for 5 seconds, then video plays in full.

In [69]:
success, output_file = run_mmmeld(["--image", f"{local_image1},{local_video_med2}", "--output", "test14.mp4"])
print(f"Test 14 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --image test_media/square.jpg,test_media/15_seconds2.mp4 --output test14.mp4 --output test_output_test14.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg

## Test 15: Multiple Images with Main Audio

Expected behavior: Equal time for each image, no looping.

In [60]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", f"{local_image1},{local_image2}", "--output", "test15.mp4"])
print(f"Test 15 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image test_media/square.jpg,test_media/16_9.png --output test15.mp4 --output test_output_test15.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-

## Test 16: Multiple Images without Main Audio

Expected behavior: 5 seconds for each image.

In [61]:
success, output_file = run_mmmeld(["--image", f"{local_image1},{local_image2}", "--output", "test16.mp4"])
print(f"Test 16 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --image test_media/square.jpg,test_media/16_9.png --output test16.mp4 --output test_output_test16.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enab

## Test 17: YouTube Audio with Local Images

Expected behavior: YouTube audio with images displayed for equal durations.

In [62]:
success, output_file = run_mmmeld(["--audio", youtube_audio, "--image", f"{local_image1},{local_image2}", "--output", "test17.mp4"])
print(f"Test 17 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio https://www.youtube.com/watch?v=dQw4w9WgXcQ --image test_media/square.jpg,test_media/16_9.png --output test17.mp4 --output test_output_test17.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enabl

## Test 18: Local Audio with YouTube Video

Expected behavior: Local audio with YouTube video (cut or looped to match audio duration).

In [63]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", youtube_video, "--output", "test18.mp4"])
print(f"Test 18 {'passed' if success else 'failed'}")
display_output(output_file)

Exception in thread Thread-123 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\jon\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1073, in _bootstrap_inner
    self.run()
  File "C:\Users\jon\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Users\jon\AppData\Local\Programs\Python\Python312\Lib\threading.py", line 1010, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\jon\AppData\Local\Programs\Python\Python312\Lib\subprocess.py", line 1597, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "c:\Users\jon\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 459: character map

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image https://www.youtube.com/watch?v=9bZkp7q19f0 --output test18.mp4 --output test_output_test18.mp4
None
2024-07-13 03:23:17,817 - INFO - Starting image input processing
2024-07-13 03:23:17,817 - INFO - Image argument provided: https://www.youtube.com/watch?v=9bZkp7q19f0
2024-07-13 03:23:17,818 - INFO - Processing image input: https://www.youtube.com/watch?v=9bZkp7q19f0
2024-07-13 03:23:17,818 - INFO - Downloading YouTube video: https://www.youtube.com/watch?v=9bZkp7q19f0
2024-07-13 03:23:20,920 - ERROR - Error downloading YouTube video with pytube: get_throttling_function_name: could not find match for multiple
2024-07-13 03:23:20,920 - INFO - Attempting to download with yt-dlp...
2024-07-13 03:23:48,654 - INFO - Image input processing complete. Total images: 1
2024-07-13 03:23:48,846 - INFO - get_audio_duration: file_path=test_media/10_seconds.wav, output='10.000340'
2024-07-13 03:23:48,888 - INFO - get_media

## Test 19: Text-to-Speech with AI-Generated Image

Expected behavior: Generated speech with AI-generated image for the full duration.

In [64]:
success, output_file = run_mmmeld(["--text", tts_text, "--image", "generate", "--output", "test19.mp4"])
print(f"Test 19 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --text To be, or not to be, that is the question. --image generate --output test19.mp4 --output test_output_test19.mp4
Generating speech for text:
To be, or not to be, that is the question.

Generating speech for chunk 1/1 with elevenlabs
Generating speech with ElevenLabs...
Using voice ID: WWr4C8ld745zI3BiA8n7
Audio generated: temp_assets\Title_Existential_Dilemma_To_Be_or_Not_to_Be.wav
Image generated and saved: temp_assets\Title_Existential_Dilemma_To_Be_or_Not_to_Be_To be_ or not to be_ that is the question..png
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-lib

## Test 20: Main Audio with Background Music

Expected behavior: Main audio plays with background music looped and faded out at the end.

In [65]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--bg-music", bg_music, "--output", "test20.mp4"])
print(f"Test 20 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image test_media/square.jpg --bg-music test_media/IndigoMirage.mp3 --output test20.mp4 --output test_output_test20.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enab

## Test 21: Custom Audio Margins

Expected behavior: 1-second lead-in and 3-second fade-out applied to the main audio.

In [66]:
success, output_file = run_mmmeld(["--audio", local_audio_short, "--image", local_image1, "--audiomargin", "5.0,5.0", "--output", "test21.mp4"])
print(f"Test 21 {'passed' if success else 'failed'}")
display_output(output_file)

python mmmeld.py --autofill --cleanup --audio test_media/10_seconds.wav --image test_media/square.jpg --audiomargin 5.0,5.0 --output test21.mp4 --output test_output_test21.mp4
ffmpeg version 7.0.1-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enabl